In [104]:
from gurobipy import *
from collections import OrderedDict
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import random
import time
import itertools
import urllib
import csv
from tsputil import *
Point = complex
City  = Point
import utils
from student_utils import *
import os
import itertools
import numpy as np

In [105]:
def solve_separation(adj_matrix, points, x_star, k, VAR_TYPE=GRB.CONTINUOUS, silent=True):
    V = range(len(points))
    Vprime = range(1,len(points))
    E = [(i,j) for i in V for j in V if adj_matrix[i][j] != 'x']
    Eprime = [(i,j) for i in Vprime for j in Vprime if adj_matrix[i][j] != 'x']
    E = tuplelist(E)
    Eprime = tuplelist(Eprime)

    m = Model("SEP")
    if silent:
        m.setParam(GRB.Param.OutputFlag,0)
    m.setParam(GRB.Param.Presolve, 0)
    m.setParam(GRB.Param.Method, 0)
    m.setParam(GRB.Param.MIPGap,1e-7)

    
    ######### BEGIN: Write here your model for Task 4
    y={}
    for i,j in Eprime:
        y[i,j]=m.addVar(lb=0.0,ub=1.0,obj=x_star[i,j],vtype=VAR_TYPE,name="y[%d,%d]" % (i,j))
    z={}
    for v in Vprime:
        if v!=k:
            z[v]=m.addVar(lb=0.0,ub=1.0,obj=-1.0,vtype=VAR_TYPE,name="z[%d]" % (v))
        else:
            z[v]=m.addVar(lb=0.0,ub=1.0,obj=0.0,vtype=VAR_TYPE,name="z[%d]" % (v))

    m.update()
    # Objective
    m.modelSense = GRB.MAXIMIZE
    
    # Constraints
    for i,j in Eprime:
        m.addConstr(y[i,j]<=z[i])
        m.addConstr(y[i,j]<=z[j])
        m.addConstr(y[i,j]>=z[i]+z[j]-1)
    
    m.addConstr(z[k]==1)

    ######### END
    m.optimize()
    
    if m.status == GRB.status.OPTIMAL:
        subtour = list(filter(lambda i: z[i].x>=0.99,z))
        return m.objVal, subtour
    else:
        print("Something wrong in solve_tsplp")
        exit(0)

In [106]:
def solve_tsp(startingPos, adjMatrix, homes, points, subtours=[], VAR_TYPE=GRB.INTEGER, silent=True):
    h = [points.index(x) for x in homes]
    V = [x for x in range(len(points))]
    E = [(i,j) for i in V for j in V if adjMatrix[i][j] != 'x'] 
    E = tuplelist(E) 
    m = Model("TSP0")
    if silent:
        m.setParam(GRB.Param.OutputFlag,0)
    m.setParam(GRB.param.Presolve, 0) # no preprocessing
    m.setParam(GRB.param.Method, 0) 
    m.setParam(GRB.param.MIPGap,1e-7)
    
    
    ######### BEGIN: Write here your model for Task 1
    x = OrderedDict()
    for (i,j) in E:
        x[i,j]=m.addVar(lb=0.0, ub=1.0, obj= float(adjMatrix[i][j]), vtype=VAR_TYPE, name="%s,%s" % (points[i],points[j]))
    # Objective
    m.modelSense = GRB.MINIMIZE
    
    ## Constraints
#     for v in V:
#         m.addConstr(quicksum(x[i,v] for i,v in E.select('*',v)) == 1,"incoming_flow_balance_%d" % v)
#         m.addConstr(quicksum(x[v,i] for v,i in E.select(v,'*')) == 1,"outgoing_flow_balance_%d" % v)
    for v in V:
        m.addConstr(quicksum(x[i,v] for i,v in E.select('*',v)) == quicksum(x[v,i] for v,i in E.select(v,'*')),"flow_balance_%d" % v)
        
    for S in subtours:
        m.addConstr(quicksum(x[i,j] for i in S for j in S if i!=j and (i,j) in x)<=len(S)-1,"tour_elim_%d" % sum(S))
    
    for v in h:
        m.addConstr(quicksum(x[i,v] for i,v in E.select('*', v)) >= 1, "yeet %d" % v)
        
    s = points.index(startingPos)
    m.addConstr(quicksum(x[s,i] for s,i in E.select(s, '*')) >= 1, "yet %d" % s)
    ######### END
    
    m.optimize()
    
    if m.status == GRB.status.OPTIMAL:
   
        return {(i,j) : x[i,j].x for i,j in x}
    else:
        print("Something wrong in solve_tsplp")
        exit(0)

In [71]:
input_data = utils.read_file('Inputs/1_50.in')
number_of_locations, number_of_houses, list_of_locations, list_of_houses, starting_location, adjacency_matrix = data_parser(input_data)
lpsol = solve_tsp(starting_location, adjacency_matrix, list_of_houses, list_of_locations, [], GRB.CONTINUOUS)

Solve TSP: the optimal objective value is 1717.33


In [107]:
def cutting_plane_alg(input_file, silent=True):
    input_data = utils.read_file(input_file)
    number_of_locations, number_of_houses, list_of_locations, list_of_houses, starting_location, adjacency_matrix = data_parser(input_data)
    points = list_of_locations
    Vprime = range(1,number_of_locations)
    subtours = []
    lenOfSubBefore = 0;
    found = True
    while found:
        lpsol = solve_tsp(starting_location, adjacency_matrix, list_of_houses, points, subtours, GRB.INTEGER)
        if len(lpsol) < 1:
            break
        found = False
        tmp_subtours = []
        best_val = float('-inf')
        for k in Vprime:
            value, subtour = solve_separation(adjacency_matrix, points,lpsol,k,GRB.INTEGER)
            best_val = value if value > best_val else best_val
            ######### BEGIN: write here the condition. Include a tollerance
            if value > .0001: 
            ######### END
                found = True
                tmp_subtours += [subtour]
        subtours += tmp_subtours
        subtours.sort()
        subtours = list(subtours for subtours,_ in itertools.groupby(subtours))
        if len(subtours) == lenOfSubBefore:
            break
        lenOfSubBefore = len(subtours)
    return lpsol

In [121]:
for key in a.keys():
    if a[key] > .9:
        print(key)

(0, 2)
(0, 28)
(0, 38)
(2, 0)
(3, 44)
(4, 0)
(6, 10)
(8, 42)
(10, 48)
(12, 34)
(14, 18)
(16, 33)
(18, 22)
(20, 21)
(21, 32)
(22, 12)
(24, 46)
(26, 39)
(27, 16)
(28, 27)
(30, 20)
(32, 6)
(33, 14)
(34, 36)
(36, 26)
(38, 0)
(39, 8)
(40, 24)
(42, 40)
(44, 30)
(46, 3)
(48, 4)


In [ ]:
a = cutting_plane_alg("Inputs/3_50.in", False)


In [ ]:
def runAllFiles(a):
#     for filename in os.listdir("Inputs/"):
#         if filename != "101_100.in":
    filename = "3_50.in"
    outputname = "Outputs/" + filename.rpartition('.')[0] + ".txt"
    filename = "Inputs/" + filename
    input_data = utils.read_file(filename)
    number_of_locations, number_of_houses, list_of_locations, list_of_houses, starting_location, adjacency_matrix = data_parser(input_data)
    f = open(outputname, "a+")
#         a = cutting_plane_alg(filename, True)
#     if len(a) < 1:
#         continue
    new_a = {}
    for key in a.keys():
        if a[key] >= .9:
            if list_of_locations[key[0]] in new_a:
                new_a[list_of_locations[key[0]]].append(list_of_locations[key[1]])
            else:
                new_a[list_of_locations[key[0]]] = [list_of_locations[key[1]]]
    f.write(starting_location)
    curr_key = starting_location
    m = max([len(x) for x in new_a.values()])
    count = 0
    while len(new_a) > 0:
        used = curr_key
        curr_key = new_a[curr_key][0]
        f.write(" ")
        f.write(curr_key)
        new_a[used].pop(0)
        if len(new_a[used]) < 1:
            new_a.pop(used)
    f.write("\n")
    f.write(str(number_of_houses))
    f.write("\n")
    for h in range(len(list_of_houses)):
        f.write(list_of_houses[h])
        f.write(" ")
        f.write(list_of_houses[h])
        if h < len(list_of_houses) - 1:
            f.write("\n")
    f.close()
            

In [ ]:
runAllFiles(a)

In [120]:
a

{(0, 1): -0.0,
 (0, 2): 1.0,
 (0, 3): -0.0,
 (0, 4): 0.0,
 (0, 5): -0.0,
 (0, 6): -0.0,
 (0, 7): -0.0,
 (0, 8): -0.0,
 (0, 9): -0.0,
 (0, 10): -0.0,
 (0, 11): -0.0,
 (0, 12): -0.0,
 (0, 13): -0.0,
 (0, 14): -0.0,
 (0, 15): -0.0,
 (0, 16): 0.0,
 (0, 17): -0.0,
 (0, 18): -0.0,
 (0, 19): -0.0,
 (0, 20): -0.0,
 (0, 21): -0.0,
 (0, 22): -0.0,
 (0, 23): -0.0,
 (0, 24): -0.0,
 (0, 25): -0.0,
 (0, 26): -0.0,
 (0, 27): -0.0,
 (0, 28): 1.0,
 (0, 29): -0.0,
 (0, 30): -0.0,
 (0, 31): -0.0,
 (0, 32): -0.0,
 (0, 33): -0.0,
 (0, 34): -0.0,
 (0, 35): -0.0,
 (0, 36): -0.0,
 (0, 37): -0.0,
 (0, 38): 1.0,
 (0, 39): -0.0,
 (0, 40): -0.0,
 (0, 41): -0.0,
 (0, 42): -0.0,
 (0, 43): -0.0,
 (0, 44): -0.0,
 (0, 45): -0.0,
 (0, 46): -0.0,
 (0, 47): -0.0,
 (0, 48): -0.0,
 (0, 49): -0.0,
 (1, 0): -0.0,
 (1, 6): -0.0,
 (1, 12): -0.0,
 (1, 18): -0.0,
 (1, 24): -0.0,
 (1, 30): -0.0,
 (1, 36): -0.0,
 (1, 42): -0.0,
 (1, 48): -0.0,
 (2, 0): 1.0,
 (2, 3): -0.0,
 (2, 6): -0.0,
 (2, 9): -0.0,
 (2, 12): 0.0,
 (2, 15): -0.0

In [92]:
for key in a.keys():
        if a[key] >= 1:
            new_a[list_of_locations[key[0]]] = list_of_locations[key[1]]

NameError: name 'new_a' is not defined